In [ ]:
import tflite_runtime

In [ ]:
tflite_model_path = "21_june_1100_params_300_epoch_97acc_.tflite"

pred_folder_path = "/home/glarus/HOOK_PROJECT/DATA_COLLECTION_HOOK/single_img_data3_exracted/all"

In [ ]:
def auto_adjust_brightness(image, target_mean):

    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)


    l_channel, a_channel, b_channel = cv2.split(lab_image)


    l_mean = np.mean(l_channel)


    target_mean = target_mean # default 150  # Adjust this value as desired

    diff = target_mean - l_mean



    adjusted_l_channel = np.clip(l_channel + diff, 0, 255).astype(np.uint8)


    adjusted_lab_image = cv2.merge((adjusted_l_channel, a_channel, b_channel))

    adjusted_bgr_image = cv2.cvtColor(adjusted_lab_image, cv2.COLOR_LAB2BGR)    
    
    return adjusted_bgr_image

In [ ]:
import glob, cv2


tflite_model_path = tflite_model_path


interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for img_path in glob.glob((pred_folder_path+"/*")):
    
    img = cv2.imread(img_path)
    
    inp_img = cv2.resize(img, (160, 120))[:,:,::-1]
    
    
    inp_img = auto_adjust_brightness(inp_img, 140)
    

    inp_img = np.expand_dims(inp_img, axis = 0)   
    
    #tflite inference
    interpreter.set_tensor(input_details[0]['index'], inp_img)
    
    interpreter.invoke()
    
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    black_img = np.zeros_like(img)
    
    cv2.putText(black_img, "TFL " + class_names[np.argmax(output_data)], (5, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(black_img, "TFL " + str(np.round(np.max(output_data)*100))+" %", (5, 80),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    img_and_pred = np.concatenate((img, black_img), axis=1)
    
    cv2.imshow('Live',img_and_pred)
    key = cv2.waitKey(0) & 0xFF
    if  key == 27:
        break
    else:
        pass
    
    
cv2.destroyAllWindows()